## CNN: Fine-Tunning com Wisard

### Imports

In [13]:
from modules import utils, globals
import torch
from modules import encoders
from binhd.classifiers import BinHD
from modules.cifake import Cifake
import wisardpkg as wp


In [2]:
import kagglehub

# # Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)

/home/marinapiragibe/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/marinapiragibe/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3


### Carregando o dataset pré-treinado

In [3]:
model = utils.load_model_from_file("resnet18_cifake_finetuned_float32.pth")
print("Modelo pré-treinado carregado.")
print(model)

/home/marinapiragibe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/marinapiragibe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Modelo 'resnet18_cifake_finetuned_float32.pth' carregado para avaliação de desempenho.
Modelo pré-treinado carregado.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn

In [4]:
import os
import random
from torch.utils.data import Subset
from torchvision import models, transforms

import torchvision


transform = transforms.Compose([
                                     transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])
                  ])

train_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join(globals.DATASET_PATH, 'train'),
    transform=transform
)

test_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join(globals.DATASET_PATH, 'test'),
    transform=transform
)

# Cria subsets para testar a lógica do modelo com um número menor do dataset (descomentar para usar)
subset_train_indices = list(range(globals.NUM_SAMPLES_TRAIN_DEBUGGER))
subset_test_indices = list(range(globals.NUM_SAMPLES_TEST_DEBUGGER))

total_train_samples = len(train_dataset)
num_train_to_select = min(globals.NUM_SAMPLES_TRAIN_DEBUGGER, total_train_samples)
subset_train_indices = random.sample(range(total_train_samples), num_train_to_select)

# Pega uma amostra aleatória de índices para o teste
total_test_samples = len(test_dataset)
num_test_to_select = min(globals.NUM_SAMPLES_TEST_DEBUGGER, total_test_samples)
subset_test_indices = random.sample(range(total_test_samples), num_test_to_select)

train_dataset_debugger = Subset(train_dataset, subset_train_indices)
test_dataset_debugger = Subset(test_dataset, subset_test_indices)

train_loader = torch.utils.data.DataLoader(
    train_dataset_debugger,
    batch_size=globals.BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset_debugger,
    batch_size=globals.BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])



In [5]:
all_features = []
all_labels = []

with torch.no_grad():
    for k, batch in enumerate(train_loader):
        print(f'\r{k+1}/{len(train_loader)}', end='', flush=True) 

        dado, rotulo = batch
        dado = dado.to(globals.DEVICE)
        rotulo = rotulo.to(globals.DEVICE)
        # Extrair features
        features = feature_extractor(dado)

        # Achatar (flatten) o tensor para (batch_size, num_features)
        features = features.view(features.size(0), -1)

        all_features.append(features.cpu())
        all_labels.append(rotulo)

print(all_features)
print(all_labels)

125/125[tensor([[1.0613, 0.4998, 0.3531,  ..., 1.4759, 1.8978, 1.2659],
        [1.2646, 0.2671, 0.8816,  ..., 0.1105, 1.1201, 3.4389],
        [0.4805, 0.3957, 0.3440,  ..., 2.3806, 0.4562, 0.8765],
        ...,
        [0.0206, 0.2321, 0.2767,  ..., 0.0138, 1.0919, 2.3167],
        [0.0474, 2.2340, 1.3046,  ..., 0.0278, 0.1435, 0.7755],
        [0.6525, 0.8840, 1.4494,  ..., 1.0487, 0.6139, 0.9029]]), tensor([[1.1572, 0.6100, 0.6203,  ..., 1.9136, 0.0196, 0.9526],
        [0.1418, 0.5060, 3.1459,  ..., 0.2495, 0.4451, 0.2233],
        [1.5305, 1.1437, 1.2321,  ..., 0.1781, 2.1453, 1.6610],
        ...,
        [1.6679, 2.4879, 0.3570,  ..., 0.5049, 0.5644, 0.5040],
        [0.6161, 0.5572, 0.5127,  ..., 0.2766, 1.6796, 0.6993],
        [1.0262, 1.5422, 1.0471,  ..., 0.0836, 0.0873, 0.4754]]), tensor([[0.7068, 0.7418, 0.9548,  ..., 0.3445, 0.6802, 0.6443],
        [1.4478, 0.5368, 0.3423,  ..., 0.1386, 1.1691, 0.3444],
        [0.4501, 0.0292, 0.7755,  ..., 0.7194, 0.0740, 0.6185],
  

In [7]:
X_cnn_features = torch.cat(all_features).numpy()  # shape: (N, D)
y_labels = torch.cat(all_labels).numpy()

In [8]:
import numpy as np
from modules.termometro import codificador_termometro

n_bits = 16
X_encoded_parts = []

# Aplica termômetro para cada feature (coluna)
for i in range(X_cnn_features.shape[1]):
    coluna = X_cnn_features[:, i]
    codificada = codificador_termometro(coluna, n_bits)
    X_encoded_parts.append(codificada)

# Concatena todas as colunas codificadas horizontalmente
X_final_binary = np.hstack(X_encoded_parts).astype(np.uint8)


In [10]:
# Etapa anterior: features extraídas da CNN e codificadas com termômetro
# Resultado: X_final_binary e y_labels

# Se os rótulos não estiverem em string ainda:
y_str = y_labels.astype(str).tolist()

print(f"\nPrimeiros 5 rótulos de y (como strings): {y_str[:5]}")

# Certifique-se de que está em tipo uint8
X_encoded_uint8 = X_final_binary.astype(np.uint8)

print(f"\nDimensão de X após todas as codificações: {X_encoded_uint8.shape}")
print(f"Tipo dos elementos de X_final_binary: {X_encoded_uint8.dtype}")
print("Primeiras 2 linhas de X_final_binary:\n", X_encoded_uint8[:2])



Primeiros 5 rótulos de y (como strings): ['1', '0', '1', '0', '0']

Dimensão de X após todas as codificações: (1000, 8192)
Tipo dos elementos de X_final_binary: uint8
Primeiras 2 linhas de X_final_binary:
 [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


In [11]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X_final_binary, y_str, test_size=0.3, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")

Tamanho do conjunto de treino (X_train): 700 amostras, 8192 bits
Tamanho do conjunto de teste (X_test): 300 amostras, 8192 bits
Tipo dos elementos de X_train: uint8


In [14]:
addressSize = 8
wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test) 


WiSARD inicializado com addressSize=8
training 700 of 70000
classifying 300 of 300


In [15]:
from sklearn.metrics import classification_report


print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       147
           1       0.95      0.82      0.88       153

    accuracy                           0.89       300
   macro avg       0.90      0.89      0.89       300
weighted avg       0.90      0.89      0.89       300

